In [1]:
import dask
import dask.threaded
import dask.multiprocessing
from dask.distributed import Client
c = Client()



/home/alberta/anaconda2/envs/pangeo-cloud/lib/python3.6/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [2]:
c


Client Scheduler: tcp://127.0.0.1:46191 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 33.66 GB


In [4]:
import xarray as xr
import numpy as np
import glob
import time
                                   

data_dir = '/home/alberta/occigen/eNATL60-BLBT02/'

tfiles = sorted(glob.glob(data_dir + '*/eNATL60-BLBT02_1h_*_gridT_20090701-20090701.nc'))
sfiles = sorted(glob.glob(data_dir + '*/eNATL60-BLBT02_1h_*_gridS_20090701-20090701.nc'))


In [5]:
def non_time_coords(ds):
    return [v for v in ds.data_vars
            if 'time_counter' not in ds[v].dims]

def drop_non_essential_vars_pop(ds):
    return ds.drop(non_time_coords(ds))

%time dst=xr.open_mfdataset(tfiles,parallel=True,concat_dim='time_counter',decode_cf=False,preprocess=drop_non_essential_vars_pop,chunks={'x':120,'y':120,'deptht':1,'time_counter':24}) 
%time dss=xr.open_mfdataset(sfiles,parallel=True,concat_dim='time_counter',decode_cf=False,preprocess=drop_non_essential_vars_pop,chunks={'x':120,'y':120,'deptht':1,'time_counter':24})


CPU times: user 3.91 s, sys: 444 ms, total: 4.36 s
Wall time: 12.8 s
CPU times: user 3.82 s, sys: 328 ms, total: 4.14 s
Wall time: 12.5 s


In [6]:
def compute_buoy(t,s):
    rau0  = 1000
    grav  = 9.81
    buoy= -1*(grav/rau0)*sigma0(t,s)
    return buoy

def sigma0(t,s):
    zrau0=1000
    zsr=np.sqrt(np.abs(s))
    zs=s
    zt=t
    zr1 = ( ( ( ( 6.536332e-9*zt-1.120083e-6 )*zt+1.001685e-4)*zt - 9.095290e-3 )*zt+6.793952e-2 )*zt+999.842594
    zr2= ( ( ( 5.3875e-9*zt-8.2467e-7 )*zt+7.6438e-5 ) *zt - 4.0899e-3 ) *zt+0.824493
    zr3= ( -1.6546e-6*zt+1.0227e-4 ) *zt-5.72466e-3
    zr4= 4.8314e-4
    sigma0=( zr4*zs + zr3*zsr + zr2 ) *zs + zr1 - zrau0
    return sigma0


def save_buoy_all():
    temp=dst.votemper
    salt=dss.vosaline
    buoy=compute_buoy(temp,salt)
    print('Save buoy to netcdf')
    buoy.rename('buoy').to_dataset().to_netcdf(path='/media/extra/DATA/eNATL60/eNATL60-BLBT02-S/eNATL60-BLBT02_y2009m07d01_buoy.nc',mode='w')

%time save_buoy_all()


Save buoy to netcdf


distributed.utils - ERROR - Worker already exists tcp://127.0.0.1:33949
Traceback (most recent call last):
  File "/home/alberta/anaconda2/envs/pangeo-cloud/lib/python3.6/site-packages/distributed/utils.py", line 713, in log_errors
    yield
  File "/home/alberta/anaconda2/envs/pangeo-cloud/lib/python3.6/site-packages/distributed/scheduler.py", line 1410, in add_worker
    raise ValueError("Worker already exists %s" % address)
ValueError: Worker already exists tcp://127.0.0.1:33949
distributed.core - ERROR - Worker already exists tcp://127.0.0.1:33949
Traceback (most recent call last):
  File "/home/alberta/anaconda2/envs/pangeo-cloud/lib/python3.6/site-packages/distributed/core.py", line 412, in handle_comm
    result = yield result
  File "/home/alberta/anaconda2/envs/pangeo-cloud/lib/python3.6/site-packages/tornado/gen.py", line 729, in run
    value = future.result()
  File "/home/alberta/anaconda2/envs/pangeo-cloud/lib/python3.6/site-packages/tornado/gen.py", line 209, in wrapper


RuntimeError: NetCDF: HDF error